In [1]:
import requests
from bs4 import BeautifulSoup

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
r = requests.get('https://mjpharm.org/archives/')

In [3]:
soup = BeautifulSoup(r.content)
a = soup.find_all('a')
pages = [a_.get('href') for a_ in a if 'previous-issues/' in a_.get('href')]
pages = sorted(list(set(pages)))
len(pages)

18

In [5]:
pages

['https://mjpharm.org/previous-issues/vol-1-no-1-2001/',
 'https://mjpharm.org/previous-issues/vol-1-no-10-2012/',
 'https://mjpharm.org/previous-issues/vol-1-no-2-2002/',
 'https://mjpharm.org/previous-issues/vol-1-no-3-2003/',
 'https://mjpharm.org/previous-issues/vol-1-no-6-208/',
 'https://mjpharm.org/previous-issues/vol-1-no-9-2011/',
 'https://mjpharm.org/previous-issues/vol-2-no-1-2015/',
 'https://mjpharm.org/previous-issues/vol-2-no-11-2014/',
 'https://mjpharm.org/previous-issues/vol-2-no-2-2016/',
 'https://mjpharm.org/previous-issues/vol-3-no-1-2017/',
 'https://mjpharm.org/previous-issues/vol-4-no-1-2018/',
 'https://mjpharm.org/previous-issues/vol-5-no-1-2019/',
 'https://mjpharm.org/previous-issues/vol-6-no-1-2020/',
 'https://mjpharm.org/previous-issues/vol-7-no-1-2021/',
 'https://mjpharm.org/previous-issues/vol-7-no-2-2021/',
 'https://mjpharm.org/previous-issues/vol-8-no-1-2022/',
 'https://mjpharm.org/previous-issues/vol-8-no-2-2022/',
 'https://mjpharm.org/previous

In [9]:
from tqdm import tqdm

urls = []
for p in tqdm(pages):
    r = requests.get(p)
    soup = BeautifulSoup(r.content)
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if a_.get('href', '').endswith('.pdf')]
    urls.extend(a)
    
urls

100%|███████████████████████████████████████████| 18/18 [00:34<00:00,  1.90s/it]


['https://mjpharm.org/wp-content/uploads/2021/02/malaysian-journal-of-pharmacy-vol-1-issue-1-may-2001.pdf',
 'https://mjpharm.org/wp-content/uploads/2021/02/malaysian-journal-of-pharmacy-2011-1-2-8.pdf',
 'https://mjpharm.org/wp-content/uploads/2021/02/malaysian-journal-of-pharmacy-2001-1-9-14.pdf',
 'https://mjpharm.org/wp-content/uploads/2021/02/malaysian-journal-of-pharmacy-2001-1-15-21.pdf',
 'https://mjpharm.org/wp-content/uploads/2021/02/malaysian-journal-of-pharmacy-2001-1-22-28.pdf',
 'https://mjpharm.org/wp-content/uploads/2021/02/malaysian-journal-of-pharmacy-2001-1-29-34.pdf',
 'https://mjpharm.org/wp-content/uploads/2021/02/vol.-1-no.-10-2012-malaysian-journal-of-pharmacy.pdf',
 'https://mjpharm.org/wp-content/uploads/2021/02/malaysian-journal-of-pharmacy-2012-10-1-14.pdf',
 'https://mjpharm.org/wp-content/uploads/2021/02/malaysian-journal-of-pharmacy-2012-10-12-27.pdf',
 'https://mjpharm.org/wp-content/uploads/2021/02/malaysian-journal-of-pharmacy-2012-10-28-37.pdf',
 'htt

In [10]:
urls = sorted(list(set(urls)))
len(urls)

90

In [11]:
!mkdir mjpharm.org

In [12]:
import os

pdfs = urls
for i in tqdm(range(len(pdfs))):
    filename = os.path.join('mjpharm.org', f'{i}.pdf')
    if os.path.exists(filename):
        continue
        
    u = pdfs[i]
        
    r = requests.get(u)
    with open(filename, 'wb') as fopen:
        fopen.write(r.content)

100%|███████████████████████████████████████████| 90/90 [00:22<00:00,  3.94it/s]


In [13]:
from glob import glob
from tika import parser
import json

TIKA_HOST = 'http://localhost:9998'

In [14]:
files = glob('mjpharm.org/*.pdf')

with open('mjpharm.org.jsonl', 'w') as fopen:
    for f in tqdm(files):
        try:
            raw_xml = parser.from_file(f, TIKA_HOST, xmlContent=True)
            body = BeautifulSoup(raw_xml['content']).find('body')
            t = '\n'.join(body.findAll(text=True))
            fopen.write(f'{json.dumps(t)}\n')
        except:
            pass

100%|███████████████████████████████████████████| 90/90 [00:08<00:00, 10.17it/s]
